In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers datasets huggingface_hub fsspec==2024.10.0 -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2024.9.0 requires fsspec==2024.9.0.*, but you have fsspec 2024.10.0 which is incompatible.


## Hugging face 사용 이유
- 여러 회사들이 개발한 모델이나 제공하는 데이터셋을 편리하게 사용하는 인터페이스 제공

In [3]:
from transformers import AutoModel, AutoTokenizer

### Bert 모델 활용 준비
- https://huggingface.co/google-bert/bert-base-uncased

In [4]:
bert_model = AutoModel.from_pretrained('google-bert/bert-base-uncased')
bert_tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
type(bert_model), type(bert_tokenizer)

(transformers.models.bert.modeling_bert.BertModel,
 transformers.models.bert.tokenization_bert_fast.BertTokenizerFast)

In [6]:
text = 'What is Huggingface Transformers ?'
encoded_input = bert_tokenizer(text, return_tensors='pt') # 학습 완료된 형태에 벡터로 결과 리턴
bert_out = bert_model(**encoded_input)

In [7]:
# type(encoded_input), encoded_input
type(bert_out), bert_out

(transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions,
 BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3009,  0.0158,  0.0698,  ..., -0.3406,  0.5976,  0.5820],
          [-0.1109,  0.0754, -0.1906,  ...,  0.2970,  0.4278, -0.0391],
          [-0.5813, -0.0042,  0.4034,  ..., -0.2549,  0.2216,  0.8121],
          ...,
          [ 0.9971,  0.3301, -0.0688,  ..., -0.4873,  0.0168, -0.0345],
          [-0.2394, -0.0573, -0.5885,  ..., -0.0415,  0.3123, -0.0288],
          [ 0.7884,  0.4039,  0.0217,  ...,  0.3869, -0.4785, -0.4116]]],
        grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-8.7247e-01, -3.1464e-01, -5.2733e-01,  7.3685e-01,  1.8460e-01,
          -1.8295e-01,  8.9985e-01,  2.9762e-01, -3.9314e-01, -9.9994e-01,
           7.1273e-02,  7.2618e-01,  9.7232e-01,  3.2968e-01,  9.1369e-01,
          -7.5043e-01, -3.5754e-01, -5.6530e-01,  2.8968e-01, -6.4040e-01,
           6.0199e-01,  9.9856e-01,  3.8675e-01,  2.9

## 모델 불러오기
- 모델 : 바디와 헤드로 구분

In [8]:
# 바디만 있는 모델
# https://huggingface.co/klue/roberta-base
roberta_model = AutoModel.from_pretrained('klue/roberta-base')

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
roberta_model

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [10]:
# 바디와 헤드가 포함된 모델
# https://huggingface.co/SamLowe/roberta-base-go_emotions
from transformers import AutoModelForSequenceClassification
roberta_go_emotions_model = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions")

config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [11]:
roberta_go_emotions_model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [12]:
# 헤드에 없는 모델을 목적에 의해 불러오기
roberta_classification_model = AutoModelForSequenceClassification.from_pretrained('klue/roberta-base')

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
roberta_classification_model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         